In [1]:
import os

In [2]:
%pwd

'/home/satyam/Documents/mlops/chicken-disease/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/home/satyam/Documents/mlops/chicken-disease'

In [5]:
import tensorflow as tf

2024-05-22 19:58:38.455747: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-22 19:58:38.465809: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-22 19:58:38.575559: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-22 19:58:44.899392: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [6]:
model = tf.keras.models.load_model("artifacts/training/model.keras")

In [19]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int
    mlflow_uri: str
    experiment_name: str
    run_name: str

In [8]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [20]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_validation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.keras",
            training_data="artifacts/data_ingestion/Chicken-fecal-images",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE,
            mlflow_uri="https://dagshub.com/satyamsc0/chicken-disease.mlflow",
            experiment_name="chicken-disease-classification",
            run_name="test-run"
        )
        return eval_config


In [10]:
from urllib.parse import urlparse

In [16]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/satyamsc0/chicken-disease.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="satyamsc0"
os.environ["MLFLOW_TRACKING_PASSWORD"]="5a7753b233a8bdd1a304ece714880751be9fda6e"

In [25]:

import mlflow
import mlflow.tensorflow

class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config
        mlflow.set_experiment(self.config.experiment_name)  # Set or create an experiment in MLflow

    def _valid_generator(self):
        datagenerator_kwargs = dict(
            rescale=1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)

    def evaluation(self):
        model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        
        # Start an MLflow run
        with mlflow.start_run(run_name=self.config.run_name):
            # Log model parameters
            mlflow.log_params({
                "batch_size": self.config.params_batch_size,
                "image_size": self.config.params_image_size,
                "model_path": str(self.config.path_of_model),
                "epochs": self.config.all_params.EPOCHS,
                "learning_rate": self.config.all_params.LEARNING_RATE,
                "number_of_classes": self.config.all_params.CLASSES
            })

            # Evaluate the model
            self.score = model.evaluate(self.valid_generator)
            
            # Log evaluation metrics
            mlflow.log_metrics({
                "loss": self.score[0],
                "accuracy": self.score[1]
            })
            
            # Log the model
            mlflow.tensorflow.log_model(model, artifact_path="model")

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)


In [26]:
try:
    config = ConfigurationManager()
    val_config = config.get_validation_config()
    evaluation = Evaluation(val_config)
    evaluation.evaluation()
    evaluation.save_score()

except Exception as e:
   raise e

[2024-05-22 22:11:14,131: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-05-22 22:11:14,138: INFO: common: yaml file: params.yaml loaded successfully]
[2024-05-22 22:11:14,140: INFO: common: created directory at: artifacts]
Found 116 images belonging to 2 classes.


/home/satyam/Documents/mlops/chicken-disease/venv/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


8/8 ━━━━━━━━━━━━━━━━━━━━ 21s 2s/step - accuracy: 0.7504 - loss: 12.2397


2024/05/22 22:11:38 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2024-05-22 22:12:18,219: INFO: common: json file saved at: scores.json]
